In [38]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import os.path
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

In [44]:
def get_all_recipes():
    item_files = [f for f in listdir('./data/TimeSeries') if isfile(join('./data/TimeSeries', f))] 
    ids = [item_file[:-5].split('_')[-1] for item_file in item_files]
    all_recipes = {}
    for item_id in tqdm(ids):
        raw_xml = load_item(item_id)
        all_recipes = {**all_recipes, **get_reagents_from_xml(raw_xml, item_id)}
    return all_recipes

def get_reagents_from_xml(raw_xml, item_id):
    """
    Get all reageants for all recipes of one item.
    """
    soup = BeautifulSoup(raw_xml, "lxml")
    # Get item name
    name = soup.find('b', class_ = 'q1')
    if name is None:
        name = soup.find('b', class_ = 'q2')
    if name is None:
        name = soup.find('b', class_ = 'q3')
    name = name.text
    # Get each recipe (spell)
    createdby = soup.find('createdby')
    if createdby is None:
        return  {item_id: (name, [])}
    spells = createdby.find_all('spell')
    # Get each reagent in each recipe
    reagent_lists = [spell.find_all('reagent') for spell in spells]
    return {item_id: (name, reagent_lists)}

def load_item(item_id):
    """
    Load an item from local or from WowHead API.
    """
    file_name = './data/item_{}.xml'.format(item_id)
    url = 'https://www.wowhead.com/item={}&xml'.format(item_id)
    if os.path.isfile(file_name):
        f = open(file_name, "r")
        raw_xml = f.read()
        f.close()
    else:
        print('Downloading item {}'.format(item_id))
        sleep(30)
        raw_xml = requests.get(url).text
        with open(file_name, 'w') as outfile:
            outfile.write(raw_xml)
    return raw_xml

In [46]:
all_recipes = get_all_recipes()

100%|██████████| 145/145 [40:08<00:00, 16.61s/it]


In [26]:
import networkx as nx

def create_graph(reagent_dict):
    G = nx.Graph()
    # add nodes for each id:
    for item in reagent_dict.keys():
        G.add_node(item, name = reagent_dict[item][0])
    for item in reagent_dict.keys():
        reagent_lists = reagent_dict[item][1]
        for reagent_list in reagent_lists:
            for reagent in reagent_list:
                component_id = reagent['id']
                G.add_edge(item, component_id)
    return G

In [35]:
G = create_graph(all_recipes)
name_dict = dict([(item_id, name[0]) for item_id, name in zip(all_recipes.keys(), all_recipes.values())])

In [47]:
print('ok')

ok
